This notebook will be mainly used for the capstone project

In [2]:
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import requests
!conda install -c conda-forge beautifulsoup4 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    soupsieve-1.9.3            |           py36_0          57 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    beautifulsoup4-4.8.0       |           py36_0         144 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         348 KB

The following NEW packages will be INSTALLED:

    soupsieve:      1.9.3-py36_0     conda-forge

The following packages will be UPDATED:

    beautifuls

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [3]:

results = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
from bs4 import BeautifulSoup
soup = BeautifulSoup(results,'html.parser')

In [5]:
#soup.prettify()

In [6]:
table = soup.find('table',{'class':'wikitable sortable'})
rows=table.findAll('tr')
# x=rows[2].find('td').contents[0]
# print(x)
# x=rows[3].find('td').contents[0]
# print(x)
rows
rows1=rows[1:]
rows1[0].find('td').contents[0]

'M1A'

In [7]:
# boroughs=[]
# postcode=[]
# neighbourhood=[]
# print("Postcode\tBoroughs\tNieghbourhood\n")
# for r in rows1:
# #     print(r)
#     for c in r:
# #         print(c.find('td').contents[0])
#         print(c)
#     print("mext")
# #         print(c.find('td').contents[0])
# #     boroughs.append(l.get('title'))
# boroughs

data1 = []
table = soup.find('table', attrs={'class':'wikitable sortable'})
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data1.append([ele for ele in cols if ele])

In [20]:
data=data1[1:]
df=pd.DataFrame(data)
df.columns = ['PostalCode', 'Borough', 'Neighborhood']
df

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West
286,M8Z,Etobicoke,South of Bloor


In [23]:
df=df[df["Borough"] != "Not assigned"]
df.reset_index(drop=True,inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [30]:
df_grouped = df.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df_grouped.head(100)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
95,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol..."
96,M9L,North York,Humber Summit
97,M9M,North York,"Emery, Humberlea"
98,M9N,York,Weston


In [31]:
for index, row in df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [32]:
df_grouped.shape

(103, 3)

In [33]:
coordinates = pd.read_csv("Geospatial_Coordinates.csv")
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [34]:
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [35]:
df_new = df_grouped.merge(coordinates, on="PostalCode", how="left")
df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [38]:
# !conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Toronto'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.49   | 

In [40]:
import folium 
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Borough'], df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

Filter only the boroughs that contain the word Totonto

In [42]:
borough_names = list(df_new.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [43]:
df_new = df_new[df_new['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(df_new.shape)
df_new.head()

(38, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [45]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Borough'], df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [46]:
# define Foursquare Credentials and Version
CLIENT_ID = 'XW1G3EUTWPVJLTVSWY4OUOUS5NRBTTBQPTZR3HK3U4W43TTH' # your Foursquare ID
CLIENT_SECRET = '0UBOQQVR30EDQ11K2SHHAVUAYGI4JSXE44ZQN3F22ESSP222' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XW1G3EUTWPVJLTVSWY4OUOUS5NRBTTBQPTZR3HK3U4W43TTH
CLIENT_SECRET:0UBOQQVR30EDQ11K2SHHAVUAYGI4JSXE44ZQN3F22ESSP222


In [47]:

radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['PostalCode'], df_new['Borough'], df_new['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [48]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1711, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [50]:

# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1711, 238)


,PostalCode,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4K,East Toronto,"The Danforth West, Riverdale",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(38, 238)


,PostalCode,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M4E,East Toronto,The Beaches,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.250000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,M4K,East Toronto,"The Danforth West, Riverdale",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.022727,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.022727
2,M4L,East Toronto,"The Beaches West, India Bazaar",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,M4M,East Toronto,Studio District,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.025641
4,M4N,Central Toronto,Lawrence Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,M4P,Central Toronto,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,M4R,Central Toronto,North Toronto West,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000
7,M4S,Central Toronto,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.027778,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,M4T,Central Toronto,"Moore Park, Summerhill East",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.250000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000


In [52]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(38, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Health Food Store,Neighborhood,Pub,Trail,Yoga Studio,Doner Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Bookstore,Indian Restaurant,Sports Bar,Spa,Brewery
2,M4L,East Toronto,"The Beaches West, India Bazaar",Sandwich Place,Pizza Place,Food & Drink Shop,Italian Restaurant,Pub,Sushi Restaurant,Brewery,Ice Cream Shop,Fish & Chips Shop,Fast Food Restaurant
3,M4M,East Toronto,Studio District,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Convenience Store,Brewery,Sandwich Place,Cheese Shop,Chinese Restaurant
4,M4N,Central Toronto,Lawrence Park,Dim Sum Restaurant,Park,Bus Line,Swim School,Yoga Studio,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
5,M4P,Central Toronto,Davisville North,Gym,Hotel,Park,Breakfast Spot,Dog Run,Clothing Store,Sandwich Place,Food & Drink Shop,Discount Store,Doner Restaurant
6,M4R,Central Toronto,North Toronto West,Coffee Shop,Sporting Goods Shop,Gym / Fitness Center,Metro Station,Mexican Restaurant,Diner,Dessert Shop,Cosmetics Shop,Bagel Shop,Clothing Store
7,M4S,Central Toronto,Davisville,Sandwich Place,Dessert Shop,Pizza Place,Gym,Coffee Shop,Café,Sushi Restaurant,Italian Restaurant,Indian Restaurant,Flower Shop
8,M4T,Central Toronto,"Moore Park, Summerhill East",Gym,Playground,Trail,Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Event Space,Department Store,Donut Shop
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",Pub,Coffee Shop,Light Rail Station,Pizza Place,Supermarket,Sushi Restaurant,Sports Bar,Fried Chicken Joint,Restaurant,Bagel Shop


In [66]:
from sklearn.cluster import KMeans
# set number of clusters

kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 1, 1, 4, 1, 1, 1, 1, 1], dtype=int32)

In [67]:

# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
toronto_merged = df_new.copy()

# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")

print(toronto_merged.shape)
toronto_merged.head() # check the last columns!
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(38, 16)
(38, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Neighborhood,Pub,Trail,Yoga Studio,Doner Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant
20,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,1,Coffee Shop,Café,Hotel,Restaurant,Bar,American Restaurant,Deli / Bodega,Bakery,Gastropub,Italian Restaurant
21,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,1,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gym,Deli / Bodega,Gastropub,Seafood Restaurant,Steakhouse
24,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,1,Coffee Shop,Sandwich Place,Café,Pizza Place,History Museum,Burger Joint,Jewish Restaurant,Pub,Indian Restaurant,BBQ Joint
25,M5S,Downtown Toronto,"Harbord, University of Toronto",43.662696,-79.400049,1,Café,Bookstore,Bar,Japanese Restaurant,Sandwich Place,Restaurant,Bakery,Dessert Shop,Chinese Restaurant,Pub
26,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049,1,Café,Vegetarian / Vegan Restaurant,Bar,Mexican Restaurant,Dumpling Restaurant,Vietnamese Restaurant,Bakery,Chinese Restaurant,Coffee Shop,Comfort Food Restaurant
27,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420,1,Airport Lounge,Airport Service,Bar,Boutique,Coffee Shop,Boat or Ferry,Harbor / Marina,Sculpture Garden,Airport Terminal,Airport Gate
19,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752,1,Coffee Shop,Hotel,Aquarium,Café,Fried Chicken Joint,Scenic Lookout,Brewery,Music Venue,Sporting Goods Shop,Bakery
28,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846,1,Coffee Shop,Restaurant,Café,Italian Restaurant,Seafood Restaurant,Cocktail Bar,Hotel,Beer Bar,Farmers Market,Fast Food Restaurant
30,M6G,Downtown Toronto,Christie,43.669542,-79.422564,1,Grocery Store,Café,Park,Baby Store,Coffee Shop,Restaurant,Bank,Nightclub,Athletics & Sports,Italian Restaurant


In [72]:
# import matplotlib.cm as cm
# import matplotlib.colors as colors

# map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# # set color scheme for the clusters
# x = np.arange(kclusters)
# ys = [i+x+(i*x)**2 for i in range(kclusters)]
# colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
# rainbow = [colors.rgb2hex(i) for i in colors_array]

# # add markers to the map
# markers_colors = []
# for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
#     label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
#     folium.CircleMarker(
#         [lat, lon],
#         radius=5,
#         popup=label,
#         color=rainbow[cluster-1],
#         fill=True,
#         fill_color=rainbow[cluster-1],
#         fill_opacity=0.7).add_to(map_clusters)
       
# map_clusters

In [70]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Health Food Store,Neighborhood,Pub,Trail,Yoga Studio,Doner Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant


In [71]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Downtown Toronto,1,Coffee Shop,Café,Hotel,Restaurant,Bar,American Restaurant,Deli / Bodega,Bakery,Gastropub,Italian Restaurant
21,Downtown Toronto,1,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gym,Deli / Bodega,Gastropub,Seafood Restaurant,Steakhouse
24,Central Toronto,1,Coffee Shop,Sandwich Place,Café,Pizza Place,History Museum,Burger Joint,Jewish Restaurant,Pub,Indian Restaurant,BBQ Joint
25,Downtown Toronto,1,Café,Bookstore,Bar,Japanese Restaurant,Sandwich Place,Restaurant,Bakery,Dessert Shop,Chinese Restaurant,Pub
26,Downtown Toronto,1,Café,Vegetarian / Vegan Restaurant,Bar,Mexican Restaurant,Dumpling Restaurant,Vietnamese Restaurant,Bakery,Chinese Restaurant,Coffee Shop,Comfort Food Restaurant
27,Downtown Toronto,1,Airport Lounge,Airport Service,Bar,Boutique,Coffee Shop,Boat or Ferry,Harbor / Marina,Sculpture Garden,Airport Terminal,Airport Gate
19,Downtown Toronto,1,Coffee Shop,Hotel,Aquarium,Café,Fried Chicken Joint,Scenic Lookout,Brewery,Music Venue,Sporting Goods Shop,Bakery
28,Downtown Toronto,1,Coffee Shop,Restaurant,Café,Italian Restaurant,Seafood Restaurant,Cocktail Bar,Hotel,Beer Bar,Farmers Market,Fast Food Restaurant
30,Downtown Toronto,1,Grocery Store,Café,Park,Baby Store,Coffee Shop,Restaurant,Bank,Nightclub,Athletics & Sports,Italian Restaurant
31,West Toronto,1,Supermarket,Pharmacy,Bakery,Gym / Fitness Center,Music Venue,Café,Middle Eastern Restaurant,Brewery,Bar,Bank
